## セクション９　SVM
- 分類と回帰どっちでも使える
- 精度が高いことで有名
- 直近のデータからある程度余裕を持った方が未知のデータにも対応できる
- あるデータがサポートとなり（サポートベクター）、そこからの距離（マージン）を最大化するように決定境界をひく
- 距離を使うので、事前にSVMでは標準化をすることが望ましい

### ソフトマージンとハードマージン
- ハードマージン：誤分類を許容しない
- ソフトマージン：誤分類を許容する（外れ値によって大きく結果が変わらない→ロバスト性が高い）  
　境界の直近のデータがサポートベクターになるとは限らない

### svmの損失関数
- マージンを最大化にしつつ誤分類を減らすmin{1/M + C*Σ(ξi)}　：i番目のデータがマージンを犯した具合の残差の合計
- パラメータ Cにより誤分類の許容を調整する
- マージンMの逆数の最小化＝マージンMの最大化

### 超平面（hyperplane）：n次の超平面の式（θ0x0+θ1x1+θ2x2+...+θnxn=0 θTX=0）
- n次元空間内の平坦なn-1次元の集合
- 線形分類器の決定境界は超平面になる 

### マージン
- 決定境界である超平面からサポートベクターまでの距離
- 先ほどの式の1/Mに当たる部分

### 損失関数ξ側:ヒンジ損失を使う
- 交差エントロピーを近似した損失で、近似により計算量が削減される（logの計算を省略）
- １＜や<-1で損失するようにすることでマージンを考慮した損失となる（ある程度余裕を持って分類できていないと正解だとしても若干の損失が発生する）

### カーネルトリック
- svmを非線形分類可能のアルゴリズムとする
- 特徴量空間を非線形変換することで非線形分離実現する
- 写像する関数をφ（x）　＝ (φ1（x）　,φ2（x）　,...φr（x）　)とする
  - xを（x、x^2）の二次元空間に写像する場合、φ（x）　＝ (φ1（x）　,φ2（x）) =(x,x^2)
  - 写像すると、損失関数を解く上で必要な各データ間の内積の計算量が膨大になってしまう。これを防ぐのがカーネルトリック
- カーネル関数K(xi,xj)に置き換えることで計算量を減らす

### PythonでのSVM
- sklearn.svm.SVC
  - C:エラーの正則化項の係数(大きいと誤分類できなくなる)、Kernel：使用するカーネル（Linear,polynomial,rbf,sigmoid tanh）
  - degree:'poly'のd（デフォは３）,gamma:'poly','rbf','sigmoid'の係数γ（'scale'(デフォ)、'auto'）
  - .support_vectors_でサポートベクトルリストを取得する
  - 回帰は、sklearn.svm.SVR

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns

ImportError: cannot import name 'DecisionBoundaryDisplay' from 'sklearn.inspection' (/opt/anaconda3/lib/python3.9/site-packages/sklearn/inspection/__init__.py)

In [2]:
# データ準備
df = sns.load_dataset('iris')
y_col = 'species'
X = df.drop(columns=[y_col])
y = df[y_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# 標準化
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
# PCA(サポート境界を可視化したいために２つの特徴量に次元を削減する)
pca = PCA(n_components=2)
X_train_pc = pca.fit_transform(X_train)
X_test_pc = pca.transform(X_test)

In [7]:
#学習
model = SVC(kernel='linear')
model.fit(X_train_pc,y_train)

#予測
y_pred = model.predict(X_test_pc)

#評価
accuracy_score(y_test,y_pred)

0.9111111111111111

In [8]:
#　サポートベクトルの確認（後分類しているものも含めている）
model.support_vectors_

array([[-2.01808086, -2.16076222],
       [-1.83691058,  0.22264198],
       [ 0.68991796,  0.71639709],
       [ 1.01841242,  0.75406273],
       [ 1.14259734,  0.50582016],
       [-0.59799672, -1.47307272],
       [ 1.00108719, -1.69126085],
       [ 0.21435381, -1.74582022],
       [ 0.76807504,  0.17465618],
       [ 0.58546256, -0.11934379],
       [-0.65005847, -1.76724016],
       [ 0.78292811,  0.42709078],
       [ 1.05347893, -1.00878745],
       [ 1.24654811,  0.22456375],
       [ 0.8060938 , -0.04480644],
       [ 0.67115053,  0.36047773],
       [ 0.83577732, -1.44733591],
       [ 1.36078095,  0.15396725],
       [ 1.18175475, -0.56575389],
       [ 0.9792362 , -0.36356179],
       [ 0.21142607, -1.52270906],
       [ 1.01399725, -0.74821922],
       [ 0.86207563, -0.06985412],
       [ 1.28474342,  0.5928925 ],
       [ 1.326855  ,  0.92380604],
       [ 1.08677516, -1.21143932],
       [ 0.82552169, -0.03239705],
       [ 1.52725209,  0.96532955],
       [ 1.01399725,

## 決定境界とサポートベクトルを可視化する
- sklearn.inspection.DecisionBoundaryDisplay
- .from_estimator(model,X)で描画

In [ ]:
# 決定境界描画
DecisionBoundaryDisplay.from_estimator(model, 
                                       X_train_pc,
                                       plot_method='contour',
                                       cmap=plt.cm.Paired,
                                       xlabel='first principal component',
                                       ylabel='second principal component')

# (PCA後の)学習データ描画
for class_, color in zip(model.classes_, 'bry'):
    idx = np.where(y_train == class_)
    plt.scatter(X_train_pc[idx, 0],
                X_train_pc[idx, 1],
                c=color,
                label=class_,
                edgecolor='black',
                s=20)

# サポートベクトル描画
plt.scatter(model.support_vectors_[:, 0],
            model.support_vectors_[:, 1],
            s=100,
            facecolor='none',
            linewidth=1,
            edgecolor='black')
plt.legend()